In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MarketSentimentAnalysis").getOrCreate()

bronze_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/data.csv",
    header=True,
    inferSchema=True
)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# Initialize Spark session
spark = SparkSession.builder.appName("MarketSentimentAnalysis").getOrCreate()

# Read the raw bronze data
bronze_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/data.csv",
    header=True,
    inferSchema=True
)

# Transform bronze → silver
silver_df = (
    bronze_df
    .withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("amount", col("amount").cast("double"))
    .dropna()
)

# Save silver layer as CSV (not parquet)
silver_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Volumes/workspace/data/bda_project/silver_data")


In [0]:
silver_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/silver_data",
    header=True,
    inferSchema=True
)
silver_df.show(5)



+--------------------+--------------------+---------+-------+------------+-------------+------------------+----------------+-------+--------------+-----------+--------------------+
|            event_id|          event_time|     city|channel|merchant_cat|      card_id|            amount|is_international|is_chip|is_contactless|label_fraud|                 key|
+--------------------+--------------------+---------+-------+------------+-------------+------------------+----------------+-------+--------------+-----------+--------------------+
|636f02e8-dd53-47e...|2025-01-25 04:12:...|Bengaluru|   ECom|     Grocery|CARD-44801393|1.5578442345277455|           false|  false|         false|      false|Bengaluru|Grocery...|
|aa52d3af-1e91-43b...|2025-01-29 14:58:...|   Mumbai|    POS|      Dining|CARD-72278854| 13.48360838468957|           false|   true|          true|      false|Mumbai|Dining|POS...|
|6f56530b-754e-4ed...|2025-01-24 05:34:...|   Mumbai|    POS|      Dining|CARD-47233054|0.72315